In [1]:
'''
MLP on CIFAR10 dataset
    63% Validation Accuracy @Epoch 100
    NOTE: Code was halted due not setting the max epoch to 100
'''

from __future__ import print_function

import time
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Dense, Dropout

from keras import backend as K
if K.backend() == 'tensorflow': #prevents out CUDA out of memory
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

stime = time.time()

batch_size = 128
num_classes = 10
epochs = 100

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train_legacy = x_train
y_train_legacy = y_train

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.reshape(50000, 3072)
x_test = x_test.reshape(10000, 3072)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

image_size = x_train.shape[1]

kernel_regularizer = l2(0.0001)
model = Sequential()
model.add(Dense(512, activation='relu',kernel_regularizer=kernel_regularizer, input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_regularizer=kernel_regularizer))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_regularizer=kernel_regularizer))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

opt = keras.optimizers.adam(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

data_augmentation = True
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    x_train = np.reshape(x_train, [-1, 32, 32, 3])
    datagen = ImageDataGenerator(
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
        fill_mode='nearest',
        horizontal_flip=True  # randomly flip images
        )


    datagen.fit(x_train)
    
    max_batches = 1000
    for e in range(epochs):
        batches = 0
        eptime = time.time()
        btime = time.time()
        for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=batch_size):
            
            x_batch = np.reshape(x_batch, [-1, 3072])
            model.fit(x_batch, y_batch, verbose=0)
            
            if batches%(max_batches//10) == 0 :
                print("Epoch %d/%d, Batch %d/%d, Time %fs/(100 steps)" % (e+1, epochs, batches, max_batches, time.time()-btime))
                btime = time.time()
            batches += 1
            if batches-1 >= max_batches:
                model.fit(x_batch, y_batch,validation_data=(x_test, y_test), verbose=1)
                break
        print("time %fs" % (time.time()-eptime))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

etime = time.time()
print('Time taken in min: ', (etime-stime)/60)


Using TensorFlow backend.


50000 train samples
10000 test samples


W0308 21:50:53.579096 140695746209600 deprecation.py:506] From /home/chico/.virtualenvs/dl4cv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                10250     
Total para

Epoch 9/200, Batch 800/1000, Time 6.402102s/(100 steps)
Epoch 9/200, Batch 900/1000, Time 6.493403s/(100 steps)
Epoch 9/200, Batch 1000/1000, Time 6.303911s/(100 steps)
Train on 128 samples, validate on 10000 samples
Epoch 1/1
128/128 [==============================] - 1s 4ms/step - loss: 1.4503 - acc: 0.5078 - val_loss: 1.4133 - val_acc: 0.5335
time 64.002394s
Epoch 10/200, Batch 0/1000, Time 0.063260s/(100 steps)
Epoch 10/200, Batch 100/1000, Time 6.244752s/(100 steps)
Epoch 10/200, Batch 200/1000, Time 6.318492s/(100 steps)
Epoch 10/200, Batch 300/1000, Time 6.231349s/(100 steps)
Epoch 10/200, Batch 400/1000, Time 6.273573s/(100 steps)
Epoch 10/200, Batch 500/1000, Time 6.215834s/(100 steps)
Epoch 10/200, Batch 600/1000, Time 6.296014s/(100 steps)
Epoch 10/200, Batch 700/1000, Time 6.287224s/(100 steps)
Epoch 10/200, Batch 800/1000, Time 6.269997s/(100 steps)
Epoch 10/200, Batch 900/1000, Time 6.277798s/(100 steps)
Epoch 10/200, Batch 1000/1000, Time 6.305770s/(100 steps)
Train on 1

...

Epoch 69/200, Batch 800/1000, Time 6.351589s/(100 steps)
Epoch 69/200, Batch 900/1000, Time 6.490607s/(100 steps)
Epoch 69/200, Batch 1000/1000, Time 6.387399s/(100 steps)
Train on 128 samples, validate on 10000 samples
Epoch 1/1
128/128 [==============================] - 1s 4ms/step - loss: 1.2270 - acc: 0.6016 - val_loss: 1.1945 - val_acc: 0.6191
time 64.166771s
Epoch 70/200, Batch 0/1000, Time 0.065667s/(100 steps)
Epoch 70/200, Batch 100/1000, Time 6.418430s/(100 steps)
Epoch 70/200, Batch 200/1000, Time 6.402650s/(100 steps)
Epoch 70/200, Batch 300/1000, Time 6.380716s/(100 steps)
Epoch 70/200, Batch 400/1000, Time 6.193431s/(100 steps)
Epoch 70/200, Batch 500/1000, Time 6.283009s/(100 steps)
Epoch 70/200, Batch 600/1000, Time 6.261611s/(100 steps)
Epoch 70/200, Batch 700/1000, Time 6.268904s/(100 steps)
Epoch 70/200, Batch 800/1000, Time 6.316630s/(100 steps)
Epoch 70/200, Batch 900/1000, Time 6.339541s/(100 steps)
Epoch 70/200, Batch 1000/1000, Time 6.270751s/(100 steps)
Train o

Epoch 79/200, Batch 800/1000, Time 6.224848s/(100 steps)
Epoch 79/200, Batch 900/1000, Time 6.344258s/(100 steps)
Epoch 79/200, Batch 1000/1000, Time 6.358792s/(100 steps)
Train on 128 samples, validate on 10000 samples
Epoch 1/1
128/128 [==============================] - 1s 4ms/step - loss: 1.2971 - acc: 0.5391 - val_loss: 1.1898 - val_acc: 0.6207
time 63.718594s
Epoch 80/200, Batch 0/1000, Time 0.063675s/(100 steps)
Epoch 80/200, Batch 100/1000, Time 6.323230s/(100 steps)
Epoch 80/200, Batch 200/1000, Time 6.347629s/(100 steps)
Epoch 80/200, Batch 300/1000, Time 6.372235s/(100 steps)
Epoch 80/200, Batch 400/1000, Time 6.268157s/(100 steps)
Epoch 80/200, Batch 500/1000, Time 6.419872s/(100 steps)
Epoch 80/200, Batch 600/1000, Time 6.370048s/(100 steps)
Epoch 80/200, Batch 700/1000, Time 6.404258s/(100 steps)
Epoch 80/200, Batch 800/1000, Time 6.348323s/(100 steps)
Epoch 80/200, Batch 900/1000, Time 6.400380s/(100 steps)
Epoch 80/200, Batch 1000/1000, Time 6.483172s/(100 steps)
Train o

Epoch 89/200, Batch 800/1000, Time 6.445877s/(100 steps)
Epoch 89/200, Batch 900/1000, Time 6.367348s/(100 steps)
Epoch 89/200, Batch 1000/1000, Time 6.410792s/(100 steps)
Train on 128 samples, validate on 10000 samples
Epoch 1/1
128/128 [==============================] - 1s 4ms/step - loss: 1.1675 - acc: 0.5938 - val_loss: 1.1965 - val_acc: 0.6197
time 64.145523s
Epoch 90/200, Batch 0/1000, Time 0.062192s/(100 steps)
Epoch 90/200, Batch 100/1000, Time 6.372454s/(100 steps)
Epoch 90/200, Batch 200/1000, Time 6.431317s/(100 steps)
Epoch 90/200, Batch 300/1000, Time 6.345437s/(100 steps)
Epoch 90/200, Batch 400/1000, Time 6.356574s/(100 steps)
Epoch 90/200, Batch 500/1000, Time 6.244078s/(100 steps)
Epoch 90/200, Batch 600/1000, Time 6.303826s/(100 steps)
Epoch 90/200, Batch 700/1000, Time 6.264653s/(100 steps)
Epoch 90/200, Batch 800/1000, Time 6.279315s/(100 steps)
Epoch 90/200, Batch 900/1000, Time 6.329405s/(100 steps)
Epoch 90/200, Batch 1000/1000, Time 6.343350s/(100 steps)
Train o

Epoch 99/200, Batch 800/1000, Time 6.261708s/(100 steps)
Epoch 99/200, Batch 900/1000, Time 6.263270s/(100 steps)
Epoch 99/200, Batch 1000/1000, Time 6.311180s/(100 steps)
Train on 128 samples, validate on 10000 samples
Epoch 1/1
128/128 [==============================] - 1s 4ms/step - loss: 1.0558 - acc: 0.6562 - val_loss: 1.1844 - val_acc: 0.6274
time 63.353742s
Epoch 100/200, Batch 0/1000, Time 0.067579s/(100 steps)
Epoch 100/200, Batch 100/1000, Time 6.222256s/(100 steps)
Epoch 100/200, Batch 200/1000, Time 6.202484s/(100 steps)
Epoch 100/200, Batch 300/1000, Time 6.229452s/(100 steps)
Epoch 100/200, Batch 400/1000, Time 6.271571s/(100 steps)
Epoch 100/200, Batch 500/1000, Time 6.330204s/(100 steps)
Epoch 100/200, Batch 600/1000, Time 6.352223s/(100 steps)
Epoch 100/200, Batch 700/1000, Time 6.309391s/(100 steps)
Epoch 100/200, Batch 800/1000, Time 6.244361s/(100 steps)
Epoch 100/200, Batch 900/1000, Time 6.307629s/(100 steps)
Epoch 100/200, Batch 1000/1000, Time 6.348714s/(100 ste

KeyboardInterrupt: 